In [ ]:
#pip install pandas numpy scikit-learn matplotlib yfinance tensorflow[and-cuda] scikeras keras_tuner seaborn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import keras_tuner as kt
import seaborn as sns
from keras_tuner import HyperModel, Hyperband

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow import keras

from graphing import graph_normal, show_results, graph_line
from results import get_num_correct_direction_difference
from get_data import get_apple_stock_split , get_recent_apple_stock_split

2025-04-07 19:28:02.333968: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 19:28:02.384176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744050482.393085   60746 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744050482.395971   60746 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744050482.403558   60746 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

''

In [2]:
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

DAYS_LAG = 14

X_train, X_test, y_train, y_test = get_apple_stock_split(DAYS_LAG)
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


X_train_scale = scaler_X.fit_transform(X_train)
y_train_scale = scaler_Y.fit_transform(y_train)

X_test_scale = scaler_X.transform(X_test)
y_test_scale = scaler_Y.transform(y_test)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Training samples: 2174, Testing samples: 544


In [3]:
#repeatable way to create the model 
def create_ann(hp):
    model = keras.Sequential([
        keras.Input(shape=(X_train.shape[1],)),  # Define input shape using Input layer
        keras.layers.Dense(
                            hp.Int('units', min_value=32, max_value=256, step=32),
                            activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])),
        keras.layers.Dense(1, activation='linear')  # Linear activation for regression
                           
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [0.001, 0.01, 0.0001])),
        loss='mse', metrics=['mae'])
    return model

In [4]:
tuner = kt.RandomSearch(
    create_ann,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=2,
    directory='tuning_dir',
    project_name='time_series_ann'
)
tuner.search(X_train_scale, y_train_scale, validation_data=(X_test_scale, y_test_scale), epochs=50, batch_size=32)


I0000 00:00:1744050507.960190   60746 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9706 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


Reloading Tuner from tuning_dir/time_series_ann/tuner0.json


In [5]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'units': 32, 'activation': 'tanh', 'learning_rate': 0.01}


In [7]:
# Get the best model found by the tuner
best_model = tuner.get_best_models(num_models=1)[0]

predictions = best_model.predict(X_test_scale).flatten()

# Predict on test data
predictions = scaler_Y.inverse_transform(final_model.predict(X_test).reshape(-1,1))
actual = scaler_Y.inverse_transform(y_test)

mae = mean_absolute_error(actual, predictions)
mse = mean_squared_error(actual, predictions)
rmse = np.sqrt(mse)

print(f"Final Model RMSE: {rmse}")
print(f"Final Model MAE: {mae}")

/mnt/c/Users/kenan/dev/university_final_year_project/venv_linux/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1744050527.270903   60866 service.cc:152] XLA service 0x7fcce00031f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744050527.270936   60866 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-04-07 19:28:47.274761: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744050527.336529   60866 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1744050527.422804   60866 device_compiler.h:188] Compiled cluster using XLA!  This line is logge

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  


ValueError: Per-column arrays must each be 1-dimensional

In [ ]:
get_num_correct_direction_difference(predictions, y_test_scale)

In [ ]:
predictions_original = scaler_Y.inverse_transform(predictions.reshape(-1, 1)).flatten()
print(predictions_original)

In [ ]:
predictions_original = scaler_Y.inverse_transform(predictions.reshape(-1, 1)).flatten()

y_test_original = scaler_Y.inverse_transform(y_test.reshape(-1, 1)).flatten()

plt.scatter(y_test, predictions, color='blue', alpha=0.5, label='Predictions vs Actual', s =5)
plt.plot([min(y_test_original), max(y_test_original)], [min(y_test_original), max(y_test_original)], color='red', linestyle='--', label='Ideal Line')

# Adding lines at y = 0 and x = 0
plt.axhline(0, color='black', linestyle='--',alpha=0.2, linewidth=1)  # Horizontal line at y = 0
plt.axvline(0, color='black', linestyle='--',alpha=0.2, linewidth=1)  # Vertical line at x = 0

# Labels and title
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predictions vs Actual Values (Original Scale)')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# trying to predict more recent ones
X, y = get_recent_apple_stock_split(DAYS_LAG)

print(f"samples: {len(X)}")

#preprocess
X_scaled = scaler_X.transform(X)
y_scaled = scaler_Y.transform(y)
predictions = grid_search.best_estimator_.predict(X_scaled)

mae = mean_absolute_error(y_scaled, predictions)
mse = mean_squared_error(y_scaled, predictions)
rmse = np.sqrt(mse)

graph_normal(predictions, y_scaled, "recent data predicitons")
graph_line(predictions, y_scaled, "recent data predicitons")
get_num_correct_direction_difference(predictions,y_scaled.reshape(-1,1))